In [1]:
!pip install langchain langchain-google-genai transformers torch pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 58.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
 

In [2]:
import os
from PIL import Image
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema.runnable import RunnableMap, RunnablePassthrough
from getpass import getpass


In [25]:
GOOGLE_API_KEY = getpass("gemini api key here bro")
llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash", google_api_key=GOOGLE_API_KEY, temperature=0)


gemini api key here bro··········


In [11]:
def generate_caption(image_path):
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    image = Image.open(image_path).convert("RGB")
    inputs = processor(image, return_tensors="pt")
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption


In [46]:
prompt_classify = PromptTemplate.from_template("""
You are an intelligent waste classification assistant.

Your task is to analyze the following description of an item: "{caption}"

Based on this description, determine the correct **type of waste** the item belongs to. Choose exactly one of the following categories:

- Biodegradable: Naturally decomposes (e.g., food waste, plant material)
- Non-biodegradable: Does not decompose easily (e.g., plastic wrappers, glass)
- Recyclable: Can be processed and reused (e.g., clean plastic bottles, cardboard, metal cans)
- Medical: Related to healthcare or personal hygiene (e.g., syringes, bandages, sanitary pads)
- Electronic: Devices or components using electricity or batteries (e.g., phones, chargers, batteries)

Respond only with the **waste type** name in lowercase.
Example responses:
- biodegradable
- recyclable
- electronic

Now classify this item.
""")


In [45]:
prompt_explain=PromptTemplate.from_template(

    """
    You are an intelligent waste classification assistant.

Your task is to tell why this waste should go in this category of dustbin color: "{caption}"
. Dustbin Color (based on Indian norms):
   - Green: Wet waste (biodegradable like food, leaves)
   - Blue: Dry waste (recyclable like plastic, paper, metal)
   - Red: BioMedical Waste like plastic having blood or infectious things
   - Yellow: Biomedical waste like syringed gloves etc
   - Black: E-waste

   If it is a sanitary waste like sanitary napkins, tampons, diapers (both adult and baby), and condoms.
   then  Red bin (preferred) or Blue bin (if red not available,must be securely wrapped before disposal)

   You just need to give 1 liner explaintion to why this waste should go in this color of dustbin


    """
)

In [58]:

prompt_bin = PromptTemplate.from_template(
    """
You are an intelligent waste classification assistant.

Given this item description: "{caption}", respond with **only** the appropriate dustbin color based on Indian waste management norms.

Allowed outputs:
- green
- blue
- red
- yellow
- black
- For sanitary waste (sanitary napkins, tampons, diapers, condoms): return exactly this → red (preferred), or blue if red is not available (must be securely wrapped)

Do not include any explanation or extra words. Only respond with the correct option from above.
"""
)


In [59]:
chain_classify = (
    RunnableMap({"caption": RunnablePassthrough()})
    | prompt_classify
    | llm
    | StrOutputParser()
)

In [60]:
chain_bin = (
    RunnableMap({"caption": RunnablePassthrough()})
    | prompt_bin
    | llm
    | StrOutputParser()
)

In [61]:
chain_explain = (
    RunnableMap({"caption": RunnablePassthrough()})
    | prompt_explain
    | llm
    | StrOutputParser()
)

In [62]:
def classify_image(image_path):

    caption = generate_caption(image_path)





    return(caption)


In [63]:
def classify_waste(image_path):

    caption = generate_caption(image_path)



    result=chain_classify.invoke(caption)

    return(result)




In [64]:
def classify_bin(image_path):

    caption = generate_caption(image_path)



    result = chain_classify.invoke(caption)

    return(result)


In [65]:
def classify_explain(image_path):

    caption = generate_caption(image_path)



    result = chain_classify.invoke(caption)

    return(result)


In [66]:
import json

def answer_dict(image_path):
    caption = generate_caption(image_path)


    category = chain_classify.invoke(caption).strip()
    bin_color = chain_bin.invoke(caption).strip()
    explanation = chain_explain.invoke(caption).strip()

    return {
        "caption": caption,
        "category": category,
        "bin": bin_color,
        "explain": explanation
    }

def answer_json(image_path):
    result = answer_dict(image_path)
    return json.dumps(result, indent=4)


In [70]:
img="/content/why-are-disposable-water-bottles-harmful-to-the-environment.webp"
print(answer_json(img))
print(answer_dict(img))

{
    "caption": "a plastic bottle sitting on the ground",
    "category": "recyclable",
    "bin": "blue",
    "explain": "The plastic bottle should go in the blue bin because it is recyclable dry waste."
}
{'caption': 'a plastic bottle sitting on the ground', 'category': 'recyclable', 'bin': 'blue', 'explain': 'The plastic bottle should go in the blue dustbin because it is recyclable dry waste.'}


In [75]:
img="/content/image;s=272x0.jpeg"
print(answer_json(img))
print(answer_dict(img))

{
    "caption": "a remote control device on a striped rug",
    "category": "electronic",
    "bin": "black",
    "explain": "The remote control should go in the black dustbin because it is e-waste."
}
{'caption': 'a remote control device on a striped rug', 'category': 'electronic', 'bin': 'black', 'explain': 'The remote control should go in the black bin because it is e-waste.'}


In [76]:
img="/content/images-4.jpeg"
print(answer_json(img))
print(answer_dict(img))

{
    "caption": "a plastic bottle on the beach",
    "category": "recyclable",
    "bin": "blue",
    "explain": "The plastic bottle should go in the blue bin because it is recyclable dry waste."
}
{'caption': 'a plastic bottle on the beach', 'category': 'recyclable', 'bin': 'blue', 'explain': 'The plastic bottle should go in the blue bin because it is recyclable dry waste.'}


In [78]:
img="/content/istockphoto-119104612-612x612.jpg"
print(answer_json(img))
print(answer_dict(img))

{
    "caption": "a red apple cut in half",
    "category": "biodegradable",
    "bin": "green",
    "explain": "A red apple cut in half goes in the green bin because it is wet, biodegradable food waste."
}
{'caption': 'a red apple cut in half', 'category': 'biodegradable', 'bin': 'green', 'explain': 'A red apple cut in half goes in the green bin because it is wet, biodegradable food waste.'}


In [79]:
img="/content/used-syringe-with-blood-on-the-asphalt-HMWNA4.jpg"
print(answer_json(img))
print(answer_dict(img))

{
    "caption": "a sy injector on the ground with a needle in it - stock image",
    "category": "medical",
    "bin": "red",
    "explain": "The syringe injector with a needle is biomedical waste and should go in the **yellow** dustbin."
}
{'caption': 'a sy injector on the ground with a needle in it - stock image', 'category': 'medical', 'bin': 'red', 'explain': 'The syringe injector with a needle is biomedical waste and should go in the **yellow** dustbin.'}
